# Predicting Hanoi, Vietnam housing price

In [ ]:
import warnings
import os
import numpy as np
import pandas as pd
import datetime 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## 1. Importing libraries, cleaning data, and removing outliers

In [ ]:
# Ignoring future warnings and deprecation warnings so as not to make the notebook full of warnings
warnings.filterwarnings("ignore")

In [ ]:
# Reading the first few lines of the dataset
df = pd.read_csv("./data.csv")
df.head() 

In [ ]:
# The length of the dataset before cleaning and removing outliers
print("The initial length of the dataset is", str(len(df)), "rows.")

### Cleaning data and removing outliers

Renaming/Translating column names

In [ ]:
df_renamed = df.rename(columns = {"Ngày":"date", "Địa chỉ":"address", "Quận":"district", 
                                  "Huyện":"ward", "Loại hình nhà ở":"type_of_housing",
                                 "Giấy tờ pháp lý":"legal_paper", "Số tầng":"num_floors",
                                 "Số phòng ngủ":"num_bed_rooms", "Diện tích":"squared_meter_area",
                                 "Dài":"length_meter", "Rộng":"width_meter", "Giá/m2":"price_in_million_per_square_meter"})
df_renamed = df_renamed.drop("Unnamed: 0", axis = 1)
df_renamed = df_renamed.dropna()
df_renamed = df_renamed.reset_index()

# The length of the dataset after dropping null values
print("The length of the dataset after dropping null values is", str(len(df_renamed)), "rows.")

Cleaning data within each columns

In [ ]:
# Remove houses with "10 plus" floors and bed rooms, since this cannot be exactly quantified
df_renamed = df_renamed[df_renamed['num_floors'] != 'Nhiều hơn 10']
df_renamed = df_renamed[df_renamed['num_bed_rooms'] != 'nhiều hơn 10 phòng']

# Clean columns and convert numerical columns to float type
df_renamed['district'] = df_renamed['district'].str.replace('Quận ','').str.strip()
df_renamed['ward'] = df_renamed['ward'].str.replace('Phường ','').str.strip()
df_renamed['num_floors'] = df_renamed['num_floors'].str.strip().astype(float)
df_renamed['num_bed_rooms'] = df_renamed['num_bed_rooms'].str.replace(' phòng','').str.strip().astype(float)
df_renamed['squared_meter_area'] = df_renamed['squared_meter_area'].str.replace(' m²','').str.strip().astype(float)
df_renamed['length_meter'] = df_renamed['length_meter'].str.replace(' m','').str.strip().astype(float)
df_renamed['width_meter'] = df_renamed['width_meter'].str.replace(' m','').str.strip().astype(float)

# Clean and convert all prices to million/m2 instead of VND/m2 or billion/m2
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²'), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²'), 'price_in_million_per_square_meter'].str.replace(' tỷ/m²','').str.replace('.','').str.replace(',','.').astype(float) * 1000
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' triệu/m²','').str.replace(',','.').astype(float)
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' đ/m²','').str.replace('.','').astype(float) * 0.000001

In [ ]:
# Create dummies for categorical columns
dummy_type_of_housing = pd.get_dummies(df_renamed.type_of_housing, prefix="housing_type")
dummy_legal_paper = pd.get_dummies(df_renamed.legal_paper, prefix="legal_paper")
dummy_district = pd.get_dummies(df_renamed.district, prefix="district")
dummy_ward = pd.get_dummies(df_renamed.ward, prefix="ward")

df_cleaned = pd.concat([df_renamed, dummy_type_of_housing, dummy_legal_paper, dummy_district, dummy_ward], axis=1)
df_cleaned = df_cleaned.drop(['index', 'date', 'address', 'district', 'ward', 'type_of_housing', 'legal_paper'], axis = 1)
df_cleaned.head()

Removing outliers using IQR method

In [ ]:
def remove_outlier_IQR(df, series):
    Q1=df[series].quantile(0.25)
    Q3=df[series].quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df[series]<(Q1-1.5*IQR)) | (df[series]>(Q3+1.5*IQR)))]
    return df_final

removed_outliers = df_cleaned
columns_to_remove_outliers = ['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter',
                              'width_meter', 'price_in_million_per_square_meter']
for column in columns_to_remove_outliers:
    removed_outliers = remove_outlier_IQR(removed_outliers, column)
    
print("The final length of the dataset is", str(len(removed_outliers)), "rows.")

## 2. EDA

### 2.1. Consider the distribution of housing in districts

In [ ]:
x = df_renamed['district'].value_counts().keys()

plt.barh(x[:10], df_renamed['district'].value_counts()[:10])

==> There is an increasing trend in the city center

### 2.2. Distribution of housing types

In [ ]:
x = df_renamed['type_of_housing'].value_counts()

plt.barh(x.keys(), x)

==> In the center, there is not much land, so alley houses are more prominent than other types.

### 2.3. The legal documents

In [ ]:
x = df_renamed['legal_paper'].value_counts()
plt.barh(x.keys(), x)

### 2.4. Distribution of number of floors, number of bedrooms, area

In [ ]:
x = df_renamed['num_floors'].value_counts()
plt.bar(x.keys()[:10], x[:10])

==> Mainly houses with 3 to 6 floors

In [ ]:
x = df_renamed['num_bed_rooms'].value_counts()
plt.bar(x.keys(), x)

==> Mainly houses have with 3 to 4 rooms

In [ ]:
x = df_renamed['squared_meter_area']
plt.hist(x, edgecolor='black', color='red', bins=np.arange(0, 150+1));

The area distribution is mainly around 30 -> 60 m2

## 3. Training a Artificial Neural Network (ANN) for Regression

Scale variables using Standard Scaler and create train-test set

In [ ]:
housing = removed_outliers

# Separate predictors and response (price) variables
X = housing.loc[:, housing.columns != 'price_in_million_per_square_meter']
y = housing[['price_in_million_per_square_meter']]
to_be_scaled = ['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter', 'width_meter']

# Initiate scaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

X_scaled = X
y_scaled = y

# Storing the fit object for reference and reverse the scaling later
PredictorScalerFit=PredictorScaler.fit(X_scaled[to_be_scaled])
TargetVarScalerFit=TargetVarScaler.fit(y_scaled)
 
# Generating the standardized values of X and y
X_scaled[to_be_scaled]=PredictorScalerFit.transform(X_scaled[to_be_scaled])
y_scaled=TargetVarScalerFit.transform(y)

X_array = np.array(X_scaled.values).astype("float32")
y_array = np.array(y_scaled).astype("float32")

X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=2032)

# Sanity check to see if all train and test arrays have correct dimensions
if X_train.shape[0] == y_train.shape[0] and X_train.shape[1] == X_test.shape[1] and X_test.shape[0] == y_test.shape[0] and y_train.shape[1] == y_test.shape[1]:
    print("All train and test sets have correct dimensions.")

In [ ]:
# Fitting the ANN to the Training set
ANN = Sequential()                
ANN.add(Dense(units=10, input_dim=X_train.shape[1], 
                kernel_initializer='normal', activation='relu'))
ANN.add(Dense(1, kernel_initializer='normal'))
ANN.compile(loss='mean_squared_error', optimizer='adam')
ANN.fit(X_train, y_train,batch_size = 30,
        epochs = 10, verbose=0)

# Generating Predictions on testing data
ANN_predictions = ANN.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
ANN_predictions = TargetVarScalerFit.inverse_transform(ANN_predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data = np.concatenate((PredictorScalerFit.inverse_transform(X_test[:,:5]), X_test[:,5:]), axis=1)

# Recreating the dataset, now with predicted price using the ANN model
TestingData = pd.DataFrame(data=Test_Data, columns=X.columns)
TestingData['Price'] = y_test_orig
TestingData['ANN_predictions'] = ANN_predictions

TestingData[['Price', 'ANN_predictions']].head()

#### Finding best parameters for the ANN using grid search

In [ ]:
# Turn off TensorFlow messages and warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"

# Create the base model
def create_regression_ANN(optimizer_trial='adam'):
    model = Sequential()
    model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=optimizer_trial)
    return model

# Creathe a dictionary for trial parameters
ANN_params = {'batch_size':[10, 20, 30, 50],
             'epochs':[10, 20, 50],
             'model__optimizer_trial':['adam', 'rmsprop']}

ANN_trial = KerasRegressor(model=create_regression_ANN, verbose=0)

# Initiate the grid search and storing best parameters for later reference
ANN_grid_search = GridSearchCV(estimator=ANN_trial, param_grid=ANN_params, 
                               cv=3, n_jobs = -1).fit(X_train, y_train, verbose=0)
ANN_best_params = ANN_grid_search.best_params_

# Showing the best parameters
ANN_best_params

#### Training the ANN model with the best parameters

In [ ]:
# Fitting the ANN to the Training set
ANN = Sequential()                
ANN.add(Dense(units=10, input_dim=X_train.shape[1], 
                kernel_initializer='normal', activation='relu'))
ANN.add(Dense(1, kernel_initializer='normal'))
ANN.compile(loss='mean_squared_error', optimizer=ANN_best_params['model__optimizer_trial'])
ANN.fit(X_train, y_train,batch_size = int(ANN_best_params['batch_size']),
        epochs = int(ANN_best_params['epochs']), verbose=0)

# Generating Predictions on testing data
ANN_predictions = ANN.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
ANN_predictions = TargetVarScalerFit.inverse_transform(ANN_predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data = np.concatenate((PredictorScalerFit.inverse_transform(X_test[:,:5]), X_test[:,5:]), axis=1)

# Recreating the dataset, now with predicted price using the ANN model
TestingData = pd.DataFrame(data=Test_Data, columns=X.columns)
TestingData['Price'] = y_test_orig
TestingData['ANN_predictions'] = ANN_predictions

TestingData[['Price', 'ANN_predictions']].head()

## 4. Training a Random Forest model for Regression

#### Finding best parameters for the Random Forest model using random search and grid search

In [ ]:
# Create a dictionary of random parameters for the model
RF_random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [int(x) for x in np.linspace(10, 100, num = 10)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

In [ ]:
# Turn off TensorFlow messages and warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"

# Create the base RF model and fit the random search
RF_regressor = RandomForestRegressor()
RF_random_search = RandomizedSearchCV(estimator=RF_regressor, param_distributions=RF_random_grid, n_iter=100, cv=5, 
                                      verbose=0, random_state=2022, n_jobs = -1).fit(X_train, np.ravel(y_train))
RF_best_params = RF_random_search.best_params_
RF_best_params

In [ ]:
# Narrowing the parameters grid based on the best parameters given by the random search, then feed the grid to a grid search
RF_param_grid = {'n_estimators': [RF_best_params['n_estimators']-100, RF_best_params['n_estimators'], RF_best_params['n_estimators']+100],
               'max_features': ['sqrt', 'log2'],
               'max_depth': [RF_best_params['max_depth'] - 10, RF_best_params['max_depth'], RF_best_params['max_depth']+10],
               'min_samples_split': [5, 10],
               'min_samples_leaf': [1, 2],
               'bootstrap': [True, False]}

In [ ]:
# Turn off TensorFlow messages and warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"

# Create another base RF model and fit the grid search
RF_regressor_2 = RandomForestRegressor()
RF_grid_search = GridSearchCV(estimator=RF_regressor_2, param_grid=RF_param_grid, 
                              cv=3, n_jobs=-1, verbose=0).fit(X_train, np.ravel(y_train))

# Showing the best parameters
RF_grid_search.best_params_

#### Training the RF model with the best parameters

In [ ]:
# Fitting a RF model with the best parameters
RF = RF_grid_search.best_estimator_

# Generating Predictions on testing data
RF_predictions = RF.predict(X_test)

# Reshaping the predictions to 2D for inverse transform
RF_predictions_reshaped = RF_predictions.reshape(-1, 1)

# Scaling the predicted Price data back to original price scale
RF_predictions_original_scale = TargetVarScalerFit.inverse_transform(RF_predictions_reshaped)

# Add predictions back to TestingData
TestingData['RF_predictions'] = RF_predictions_original_scale.flatten()

# Display the results
print(TestingData[['Price', 'ANN_predictions', 'RF_predictions']].head())


## 5. Training a KNN model

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

knn = KNeighborsRegressor(n_neighbors=5)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


#### Training the KNN model with the best parameters

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2] 
}

grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']
print("Optimal number of neighbors:", best_k)

knn_optimized = KNeighborsRegressor(n_neighbors=best_k)
knn_optimized.fit(X_train, y_train)
y_pred_optimized = knn_optimized.predict(X_test)

mse_optimized = mean_squared_error(y_test, y_pred_optimized)
print("Optimized Mean Squared Error:", mse_optimized)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred_optimized)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual vs Predicted Prices (KNN)")
plt.show()

## 6. Training a Decision Tree Regressor Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Initialize Decision Tree Regressor
tree_model = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model.fit(X_train, y_train)

# Predict
y_pred = tree_model.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2 Score: {r2}")

#### Training with best parameters

In [ ]:
# Define hyperparameter grid
param_grid = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"]
}

# Use GridSearchCV for tuning
grid_search = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=1
)

# Train with grid search
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Best model
best_tree_model = grid_search.best_estimator_

# Evaluate the tuned model
y_pred_tuned = best_tree_model.predict(X_test)
mse_tuned = mean_squared_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

print(f"Tuned Mean Squared Error (MSE): {mse_tuned}")
print(f"Tuned R^2 Score: {r2_tuned}")

## Final result: Artifical Neural Network vs. Random Forest

In [ ]:
# Define a function evaluate the predictions
def Accuracy_Score(orig, pred):
    MAPE = np.mean(100 * (np.abs(orig - pred) / orig))
    return(100-MAPE)

In [ ]:
# Showing scores for both the ANN and the RF model
print("Accuracy for the ANN model is:", str(Accuracy_Score(TestingData['Price'], TestingData['ANN_predictions'])))
print("Accuracy for the RF model is:", str(Accuracy_Score(TestingData['Price'], TestingData['RF_predictions'])))